In [130]:
import numpy as np
from scipy.fft import ifft2

In [131]:
def qpsk_mapper(bits):
    bit_pairs = bits.reshape(-1, 2)
    symbols = (1 - 2 * bit_pairs[:, 0]) + 1j * (1 - 2 * bit_pairs[:, 1])
    return symbols

In [132]:
print(qpsk_mapper(np.array([0, 1, 1, 0, 1, 1, 0, 0])))

[ 1.-1.j -1.+1.j -1.-1.j  1.+1.j]


In [133]:
def qpsk_demapper(symbols):
    bits = np.zeros(2 * len(symbols), dtype=int)
    bits[0::2] = symbols.real < 0  # MSB decision (Real part)
    bits[1::2] = symbols.imag < 0  # LSB decision (Imaginary part)
    return bits

In [134]:
print(qpsk_demapper(qpsk_mapper(np.array([0,1,1,0,1,1,0,0]))))

[0 1 1 0 1 1 0 0]


In [135]:
def apply_isfft(symbols, M, N):
    # Reshape input symbols into M x N matrix
    X = symbols.reshape(M, N)
    
    # Apply 2D inverse FFT (ISFFT)
    transformed_output = ifft2(X)
    
    return transformed_output

# Example Test
M, N = 4, 4  # Example OTFS frame size
qpsk_symbols = np.random.choice([1+1j, 1-1j, -1+1j, -1-1j], size=(M * N))  # Random QPSK symbols
isfft_output = apply_isfft(qpsk_symbols, M, N)
print(isfft_output)

[[ 0.   -0.125j  0.5  +0.375j -0.5  +0.375j  0.   -0.125j]
 [-0.125-0.25j  -0.375+0.j    -0.125+0.25j   0.125+0.j   ]
 [ 0.   -0.375j  0.   +0.125j  0.   -0.375j  0.   +0.125j]
 [-0.375+0.25j  -0.125+0.5j    0.125+0.25j  -0.125+0.j   ]]


In [136]:
def apply_heisenberg_transform(isfft_output):
    # Flatten the matrix to a 1D array and apply IFFT
    transformed_output = np.fft.ifft(isfft_output.flatten())

    return transformed_output

# Example Test
M, N = 4, 4  # Example OTFS frame size
qpsk_symbols = np.random.choice([1+1j, 1-1j, -1+1j, -1-1j], size=(M * N))  # Random QPSK symbols
isfft_output = apply_isfft(qpsk_symbols, M, N)
heisenberg_output = apply_heisenberg_transform(isfft_output)

print("\nISFFT Output:\n", isfft_output)
print("\nHeisenberg Transform Output:\n", heisenberg_output)


ISFFT Output:
 [[ 0.   +0.j     0.   +0.j    -0.25 +0.25j   0.25 -0.25j ]
 [-0.125-0.625j -0.125-0.375j -0.625-0.125j -0.125+0.125j]
 [ 0.   -0.25j   0.25 +0.j    -0.25 +0.5j    0.   -0.25j ]
 [ 0.125+0.375j -0.125-0.125j  0.125-0.125j -0.125-0.125j]]

Heisenberg Transform Output:
 [-0.0625    -0.0625j      0.10669417-0.04419417j -0.0625    +0.08838835j
 -0.12542718-0.03167718j  0.0625    -0.0625j      0.01830583+0.04419417j
  0.0625    +0.j         -0.02349053+0.07025947j -0.0625    +0.0625j
  0.10669417-0.04419417j -0.0625    -0.08838835j -0.04376699+0.04998301j
  0.0625    -0.0625j      0.01830583+0.04419417j  0.0625    +0.j
 -0.05731529+0.03643471j]


In [137]:
import numpy as np
from scipy.fft import ifft2, fft2

def apply_isfft(symbols, M, N):
    """
    Applies Inverse Symplectic Finite Fourier Transform (ISFFT) on input symbols.

    Parameters:
    symbols (numpy array): Input complex symbols (M*N elements).
    M (int): Number of delay bins.
    N (int): Number of Doppler bins.

    Returns:
    numpy array: M x N matrix after ISFFT.
    """
    X = symbols.reshape(M, N)  # Reshape to M x N matrix
    transformed_output = ifft2(X)  # Apply 2D inverse FFT (ISFFT)
    
    return transformed_output

def apply_heisenberg_transform(isfft_output):
    """
    Applies Heisenberg transform to convert ISFFT output to time domain.

    Parameters:
    isfft_output (numpy array): M x N ISFFT output matrix.

    Returns:
    numpy array: 1D time-domain sequence (M*N elements).
    """
    return np.fft.ifft(isfft_output.flatten())  # Apply 1D IFFT

def apply_wigner_transform(heisenberg_output, M, N):
    """
    Applies Wigner transform (SFFT) to convert time-domain signal back to delay-Doppler domain.

    Parameters:
    heisenberg_output (numpy array): 1D time-domain sequence (M*N elements).
    M (int): Number of delay bins.
    N (int): Number of Doppler bins.

    Returns:
    numpy array: M x N Wigner transformed matrix.
    """
    reshaped_input = heisenberg_output.reshape(M, N)  # Reshape to M x N matrix
    transformed_output = fft2(reshaped_input)  # Apply 2D FFT (SFFT)
    
    return transformed_output

def apply_sfft(wigner_output):
    """
    Applies Symplectic Finite Fourier Transform (SFFT) on input data.

    Parameters:
    wigner_output (numpy array): M x N Wigner transformed matrix.

    Returns:
    numpy array: M x N matrix after applying SFFT.
    """
    transformed_output = fft2(wigner_output)  # Apply 2D FFT (SFFT)
    return transformed_output

def apply_qpsk_demodulation_to_bitstream(sfft_output):
    """
    Demodulates QPSK symbols and returns a 1D binary sequence.

    Parameters:
    sfft_output (numpy array): M x N matrix containing received complex symbols.

    Returns:
    numpy array: 1D binary sequence (length M*N*2).
    """
    qpsk_mapping = {
        1+1j: [0, 0],
        1-1j: [0, 1],
        -1+1j: [1, 0],
        -1-1j: [1, 1]
    }

    # Find the closest QPSK symbol and map it to bits
    demodulated_bits = np.array([
        qpsk_mapping[min(qpsk_mapping.keys(), key=lambda x: abs(x - val))]
        for val in sfft_output.flatten()
    ]).flatten()  # Convert to 1D binary sequence

    return demodulated_bits

# Example Test
M, N = 4, 4  # Example OTFS frame size
qpsk_symbols = np.random.choice([1+1j, 1-1j, -1+1j, -1-1j], size=(M * N))  # Random QPSK symbols

# Step 1: Apply ISFFT
isfft_output = apply_isfft(qpsk_symbols, M, N)

# Step 2: Apply Heisenberg Transform
heisenberg_output = apply_heisenberg_transform(isfft_output)

# Step 3: Apply Wigner Transform (SFFT)
wigner_output = apply_wigner_transform(heisenberg_output, M, N)

# Step 4: Apply SFFT
sfft_output = apply_sfft(wigner_output=wigner_output)

# Step 5: Apply QPSK Demodulation
demodulated_symbols = apply_qpsk_demodulation_to_bitstream(sfft_output)

# Print Outputs
print("\nInput QPSK Symbols Matrix:\n", qpsk_symbols.reshape(M, N))
print("\nISFFT Output (M x N):\n", isfft_output)
print("\nHeisenberg Transform Output (Flattened Time-Domain Signal):\n", heisenberg_output)
print("\nWigner Transform Output (M x N Matrix):\n", wigner_output)
print("\nSFFT Output (M x N Matrix):\n", sfft_output)
print("\nQPSK Demodulated Symbols (M x N Matrix):\n", demodulated_symbols)


Input QPSK Symbols Matrix:
 [[-1.+1.j -1.-1.j -1.-1.j -1.-1.j]
 [ 1.+1.j  1.+1.j -1.+1.j -1.+1.j]
 [ 1.-1.j -1.-1.j -1.+1.j -1.+1.j]
 [ 1.+1.j -1.-1.j  1.+1.j  1.+1.j]]

ISFFT Output (M x N):
 [[-0.25+0.25j  0.5 +0.j    0.25+0.25j  0.  +0.j  ]
 [-0.25-0.25j -0.5 +0.25j  0.  +0.j    0.25+0.5j ]
 [-0.5 -0.5j   0.  +0.j    0.  +0.j    0.  +0.j  ]
 [ 0.  +0.j    0.  +0.25j -0.25+0.25j -0.25+0.j  ]]

Heisenberg Transform Output (Flattened Time-Domain Signal):
 [-0.0625    +0.0625j      0.03125   +0.04161942j  0.05713835+0.07544417j
 -0.05774247+0.12024247j -0.0625    -0.0625j      0.03125   +0.06871601j
 -0.03125   +0.01294417j -0.02391771+0.08641771j -0.0625    -0.0625j
  0.03125   +0.10926893j -0.11963835-0.01294417j  0.05774247+0.00475753j
 -0.0625    -0.0625j      0.03125   -0.09460436j -0.03125   -0.07544417j
  0.02391771+0.03858229j]

Wigner Transform Output (M x N Matrix):
 [[-0.25      +2.50000000e-01j -0.25      -2.50000000e-01j
  -0.5       -5.00000000e-01j  0.        +0.00000000

In [138]:
input_bits = np.random.randint(0, 2, size=32)
qpsk_modulate = qpsk_mapper(input_bits)
isfft_output = apply_isfft(qpsk_modulate, 4, 4)
heisenberg_output = apply_heisenberg_transform(isfft_output=isfft_output)
wigner_output = apply_wigner_transform(heisenberg_output=heisenberg_output, M=4, N=4)
sfft_output = apply_sfft(wigner_output=wigner_output)
output_bits = apply_qpsk_demodulation_to_bitstream(sfft_output=sfft_output)
print(input_bits)
print(output_bits)

[1 1 1 1 1 1 0 0 1 1 1 0 0 0 0 0 1 0 1 1 1 0 1 0 0 1 1 1 1 0 0 0]
[1 1 0 0 1 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0]
